# Qwen3-TTS 파인튜닝 - 원인 분석 테스트

**v6 성공 설정:** flash_attention_2 + lr 1e-6 + batch 8

**테스트 목표:** 어떤 요소가 핵심이었는지 파악

| 테스트 | attn | lr | batch | 결과 |
|--------|------|-----|-------|------|
| v6 | flash | 1e-6 | 8 | 성공 |
| A | eager | 1e-6 | 8 | ? |
| B | flash | 2e-5 | 8 | ? |
| C | flash | 1e-6 | 2 | ? |

## 셀 1: 환경 설정 (공통)

In [ ]:
!apt-get install -y sox
!pip install -q soundfile librosa tqdm huggingface_hub
!pip install flash-attn --no-build-isolation
!git clone https://github.com/QwenLM/Qwen3-TTS.git /content/Qwen3-TTS-repo
%cd /content/Qwen3-TTS-repo
!pip install -e .
print("환경 설정 완료!")

## 셀 2: Google Drive 마운트 (공통)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("드라이브 마운트 완료!")

## 셀 3: 모델 다운로드 (공통)

In [ ]:
from huggingface_hub import snapshot_download

model_path = snapshot_download(
    "Qwen/Qwen3-TTS-12Hz-1.7B-Base", 
    local_dir="/content/qwen3_tts_model"
)
print(f"모델 준비 완료: {model_path}")

## 셀 4: 오디오 변환 + JSONL 생성 (공통)

In [ ]:
import os, json, librosa, soundfile as sf
from tqdm import tqdm

# 오디오 24kHz 변환
AUDIO_DIR = "/content/drive/MyDrive/debi_tts_data/audio"
OUTPUT_DIR = "/content/audio_24k"
os.makedirs(OUTPUT_DIR, exist_ok=True)

files = [f for f in os.listdir(AUDIO_DIR) if f.endswith('.wav')]
for f in tqdm(files):
    try:
        audio, _ = librosa.load(os.path.join(AUDIO_DIR, f), sr=24000)
        sf.write(os.path.join(OUTPUT_DIR, f), audio, 24000)
    except:
        pass

# JSONL 생성
with open("/content/drive/MyDrive/debi_tts_data/debi_finetune.jsonl", 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

valid_files = set(os.listdir("/content/audio_24k"))
REF_AUDIO = "/content/audio_24k/Debi_airSupply_2_01.wav"

filtered = []
for item in data:
    filename = item['audio'].split('/')[-1]
    if filename in valid_files:
        item['audio'] = f"/content/audio_24k/{filename}"
        item['ref_audio'] = REF_AUDIO
        filtered.append(item)

with open("/content/debi_24k.jsonl", 'w', encoding='utf-8') as f:
    for item in filtered:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"준비 완료: {len(filtered)}개 샘플")

## 셀 5: 데이터 토큰화 (공통)

In [ ]:
!python /content/Qwen3-TTS-repo/finetuning/prepare_data.py \
    --device cuda:0 \
    --tokenizer_model_path Qwen/Qwen3-TTS-Tokenizer-12Hz \
    --input_jsonl /content/debi_24k.jsonl \
    --output_jsonl /content/debi_tokenized.jsonl

---
# 테스트 A: eager + lr 1e-6 + batch 8
**가설: flash_attention이 핵심이다**

In [ ]:
# 테스트 A: eager로 패치
sft_path = "/content/Qwen3-TTS-repo/finetuning/sft_12hz.py"

with open(sft_path, 'r') as f:
    code = f.read()

code = code.replace('log_with="tensorboard"', 'log_with=None')
code = code.replace('attn_implementation="flash_attention_2"', 'attn_implementation="eager"')

with open(sft_path, 'w') as f:
    f.write(code)

print("테스트 A 패치: eager")

In [ ]:
# 테스트 A: 학습
!rm -rf /content/drive/MyDrive/debi_model_test_A
!python /content/Qwen3-TTS-repo/finetuning/sft_12hz.py \
    --init_model_path /content/qwen3_tts_model \
    --train_jsonl /content/debi_tokenized.jsonl \
    --output_model_path /content/drive/MyDrive/debi_model_test_A \
    --batch_size 8 \
    --lr 1e-6 \
    --num_epochs 5 \
    --speaker_name debi

In [ ]:
# 테스트 A: 결과 확인
import torch
from IPython.display import Audio, display
from qwen_tts import Qwen3TTSModel

tts = Qwen3TTSModel.from_pretrained(
    "/content/drive/MyDrive/debi_model_test_A/checkpoint-epoch-4",
    device_map="cuda:0",
    dtype=torch.bfloat16,
    attn_implementation="eager",
)

wavs, sr = tts.generate_custom_voice(
    text="안녕하세요, 데비입니다! 테스트 A 결과입니다.",
    speaker="debi",
)
display(Audio(wavs[0], rate=sr))
print("테스트 A 완료! (eager + lr 1e-6 + batch 8)")

---
# 테스트 B: flash + lr 2e-5 + batch 8
**가설: 초저학습률이 핵심이다**

**런타임 재시작 후 셀 1~5 다시 실행**

In [ ]:
# 테스트 B: flash 유지 패치
sft_path = "/content/Qwen3-TTS-repo/finetuning/sft_12hz.py"

with open(sft_path, 'r') as f:
    code = f.read()

code = code.replace('log_with="tensorboard"', 'log_with=None')
# flash_attention_2 유지 (수정 안 함)

with open(sft_path, 'w') as f:
    f.write(code)

print("테스트 B 패치: flash_attention_2 유지")

In [ ]:
# 테스트 B: 학습 (lr 2e-5로 변경)
!rm -rf /content/drive/MyDrive/debi_model_test_B
!python /content/Qwen3-TTS-repo/finetuning/sft_12hz.py \
    --init_model_path /content/qwen3_tts_model \
    --train_jsonl /content/debi_tokenized.jsonl \
    --output_model_path /content/drive/MyDrive/debi_model_test_B \
    --batch_size 8 \
    --lr 2e-5 \
    --num_epochs 5 \
    --speaker_name debi

In [ ]:
# 테스트 B: 결과 확인
import torch
from IPython.display import Audio, display
from qwen_tts import Qwen3TTSModel

tts = Qwen3TTSModel.from_pretrained(
    "/content/drive/MyDrive/debi_model_test_B/checkpoint-epoch-4",
    device_map="cuda:0",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

wavs, sr = tts.generate_custom_voice(
    text="안녕하세요, 데비입니다! 테스트 B 결과입니다.",
    speaker="debi",
)
display(Audio(wavs[0], rate=sr))
print("테스트 B 완료! (flash + lr 2e-5 + batch 8)")

---
# 테스트 C: flash + lr 1e-6 + batch 2
**가설: batch_size가 핵심이다**

**런타임 재시작 후 셀 1~5 다시 실행**

In [ ]:
# 테스트 C: flash 유지 패치
sft_path = "/content/Qwen3-TTS-repo/finetuning/sft_12hz.py"

with open(sft_path, 'r') as f:
    code = f.read()

code = code.replace('log_with="tensorboard"', 'log_with=None')
# flash_attention_2 유지 (수정 안 함)

with open(sft_path, 'w') as f:
    f.write(code)

print("테스트 C 패치: flash_attention_2 유지")

In [ ]:
# 테스트 C: 학습 (batch 2로 변경)
!rm -rf /content/drive/MyDrive/debi_model_test_C
!python /content/Qwen3-TTS-repo/finetuning/sft_12hz.py \
    --init_model_path /content/qwen3_tts_model \
    --train_jsonl /content/debi_tokenized.jsonl \
    --output_model_path /content/drive/MyDrive/debi_model_test_C \
    --batch_size 2 \
    --lr 1e-6 \
    --num_epochs 5 \
    --speaker_name debi

In [ ]:
# 테스트 C: 결과 확인
import torch
from IPython.display import Audio, display
from qwen_tts import Qwen3TTSModel

tts = Qwen3TTSModel.from_pretrained(
    "/content/drive/MyDrive/debi_model_test_C/checkpoint-epoch-4",
    device_map="cuda:0",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

wavs, sr = tts.generate_custom_voice(
    text="안녕하세요, 데비입니다! 테스트 C 결과입니다.",
    speaker="debi",
)
display(Audio(wavs[0], rate=sr))
print("테스트 C 완료! (flash + lr 1e-6 + batch 2)")

---
# 결과 정리

| 테스트 | attn | lr | batch | 결과 |
|--------|------|-----|-------|------|
| v6 | flash | 1e-6 | 8 | 성공 |
| A | eager | 1e-6 | 8 | |
| B | flash | 2e-5 | 8 | |
| C | flash | 1e-6 | 2 | |

**결론:**
- A 실패 → flash_attention이 핵심
- B 실패 → 초저학습률(1e-6)이 핵심
- C 실패 → batch_size 8이 핵심